Note: images was cleaned by LabelStudio anntotation

In [1]:
import sys
sys.path.append('/app')

In [2]:
import pandas as pd
from pathlib import Path
import shutil
import memlish
from memlish.io.image import load_image

In [15]:
IMGFLIP_DIR = Path('/data/imgflip/') 
IMAGE_DIR = IMGFLIP_DIR / 'images'
V1_IMAGE_DIR = IMGFLIP_DIR / 'v1_images'

V1_IMAGE_DIR.mkdir(parents=True, exist_ok=True)

## Prepare v1_images from cleaned by LabelStudio images

In [4]:
export_df = pd.read_csv('https://s3.eu-west-1.amazonaws.com/memlish.head.public/LabelStudio/memlish/v1_1800_cleaned_templates.csv')

In [5]:
approved_mask = export_df['choice'].isna()

approved_templates_ds = export_df[approved_mask].copy()

approved_templates_ds.head(2)

,image,id,choice,annotator,annotation_id,created_at,updated_at,lead_time
3,/data/upload/1/c28c65bf-Zorg.jpg,1843,NaN,1,1843,2022-02-19T22:24:02.204753Z,2022-02-19T22:24:02.204777Z,8.966
4,/data/upload/1/0a80358e-Zombie-Overly-Attached...,1842,NaN,1,1842,2022-02-19T22:23:52.698633Z,2022-02-19T22:23:52.698654Z,1.014


In [6]:
approved_templates_ds.shape

(1184, 8)

In [7]:
# LabelStudio insert uniq id to start of filename, I've removed it
labeled_images = approved_templates_ds['image'].apply(lambda x: '-'.join(x.split('-')[1:])).values

In [8]:
approved_images = [IMAGE_DIR / i for i in labeled_images]

In [9]:
approved_images[:2]

[PosixPath('/data/imgflip/images/Zorg.jpg'),
 PosixPath('/data/imgflip/images/Zombie-Overly-Attached-Girlfriend.jpg')]

In [13]:
for i in approved_images:
    shutil.copyfile(i, V1_IMAGE_DIR / i.name)

len(V1_IMAGE_DIR.ls())

1184

## Merge approved images with texts csv

In [16]:
texts_df = pd.read_csv(IMGFLIP_DIR / 'template_texts.csv')

In [18]:
image_names = [i.name for i in V1_IMAGE_DIR.ls()]

In [20]:
approved_text_mask = texts_df['template_img_name'].apply(lambda x: x in image_names)

In [22]:
df = texts_df[approved_text_mask].copy()

In [23]:
df

,url,text,alt,views,upvotes,comments,meme_home_page,template_name,template_img_name
59264,i.imgflip.com/61wlf5.jpg,ME; THE MOBILE GAME I BOUGHT 2 MINUTES AGO; TH...,Is this not true? | ME; THE MOBILE GAME I BOU...,704,6,0,/i/61wlf5,Distracted-Boyfriend,Distracted-Boyfriend.jpg
59265,i.imgflip.com/61zv6o.jpg,NEWS; GOOD THINGS IN THE WORLD; BAD THINGS IN ...,Distracted Boyfriend | NEWS; GOOD THINGS IN T...,945,6,0,/i/61zv6o,Distracted-Boyfriend,Distracted-Boyfriend.jpg
59266,i.imgflip.com/6256vy.jpg,ALSO ME; ALSO ME; ME,Distracted Boyfriend | ALSO ME; ALSO ME; ME |...,44,3,3,/i/6256vy,Distracted-Boyfriend,Distracted-Boyfriend.jpg
59267,i.imgflip.com/61z0ba.jpg,STUDY; ME; MOBILE,Distracted Boyfriend | STUDY; ME; MOBILE | im...,443,3,0,/i/61z0ba,Distracted-Boyfriend,Distracted-Boyfriend.jpg
59268,i.imgflip.com/624obq.jpg,Fortnite; Fortnite youtubers when the game is ...,Distracted Boyfriend | Fortnite; Fortnite you...,110,2,0,/i/624obq,Distracted-Boyfriend,Distracted-Boyfriend.jpg
...,...,...,...,...,...,...,...,...,...
1439254,i.imgflip.com/e8gl3.jpg,TELLS PEOPLE TO DRIVE SLOWLY IN SNOWY CONDITIO...,Scumbag Redditor | TELLS PEOPLE TO DRIVE SLOWL...,951,0,0,/i/e8gl3,Scumbag-Redditor,Scumbag-Redditor.jpg
1439255,i.imgflip.com/7tpr0.jpg,SHOWS MY POSTS ON THE WEBSITE NOT ON THE MOBIL...,Scumbag Redditor | SHOWS MY POSTS ON THE WEBSI...,561,0,0,/i/7tpr0,Scumbag-Redditor,Scumbag-Redditor.jpg
1439256,i.imgflip.com/7bakl.jpg,CRITICISES MEDIA ABOUT NOT GIVING ENOUGH COVER...,Scumbag Redditor | CRITICISES MEDIA ABOUT NOT ...,3400,0,0,/i/7bakl,Scumbag-Redditor,Scumbag-Redditor.jpg
1439257,i.imgflip.com/12qpk9.jpg,COMES TO YOUR SUB AND SAYS YOU SHOULDN'T USE A...,Scumbag Redditor | COMES TO YOUR SUB AND SAYS...,738,0,0,/i/12qpk9,Scumbag-Redditor,Scumbag-Redditor.jpg


## Deduplication

In [27]:
# install bo4ka
!pip install git+https://<git_token>@github.com/memlish/bo4ka.git --no-cache-dir

  Cloning https://****@github.com/memlish/bo4ka.git to /tmp/pip-req-build-3a1ndsjf
  Running command git clone -q 'https://****@github.com/memlish/bo4ka.git' /tmp/pip-req-build-3a1ndsjf
     |████████████████████████████████| 60.4 MB 43.6 MB/s eta 0:00:01
     |████████████████████████████████| 11.2 MB 27.3 MB/s eta 0:00:01
     |████████████████████████████████| 292 kB 137.4 MB/s eta 0:00:01
     |████████████████████████████████| 1.6 MB 42.1 MB/s eta 0:00:01
     |████████████████████████████████| 4.1 MB 124.8 MB/s eta 0:00:01
     |████████████████████████████████| 474 kB 124.8 MB/s eta 0:00:01
     |████████████████████████████████| 452 kB 133.6 MB/s eta 0:00:01
     |████████████████████████████████| 401 kB 138.3 MB/s eta 0:00:01
     |████████████████████████████████| 13.5 MB 126.0 MB/s eta 0:00:01
     |████████████████████████████████| 525 kB 127.5 MB/s eta 0:00:01
     |████████████████████████████████| 145 kB 136.5 MB/s eta 0:00:01
     |████████████████████████████████| 102 

In [32]:
import bo4ka
from bo4ka.embeddings.generators import WangHelsingEmbeddingGenerator, WangHelsingConfig
from bo4ka.matching.pipelines.p00_wanghelsing_loftr import Pipeline_00_WangHelsing_LoFTR
from bo4ka.nn.loftr import LoFTR

In [35]:
cfg = WangHelsingConfig(batch_size=16, num_workers=0, device='cpu')

wang_helsing = WangHelsingEmbeddingGenerator(V1_IMAGE_DIR, cfg=cfg)

In [36]:
image_files = [i.name for i in V1_IMAGE_DIR.ls()]

In [38]:
embs_filenames, embs = wang_helsing.get_embeddings(image_files)

Calculate descriptors:   0%|          | 0/2 [00:00<?, ?it/s]

imsize=768:   0%|          | 0/74 [00:00<?, ?it/s]

imsize=128:   0%|          | 0/74 [00:00<?, ?it/s]

In [44]:
embs_filenames = [V1_IMAGE_DIR/f for f in embs_filenames]
matcher = LoFTR(device='cpu')

pipeline = Pipeline_00_WangHelsing_LoFTR(
    template_files=embs_filenames,
    template_desc=embs,
    embedding_generator=wang_helsing,
    loftr=matcher)

In [46]:
matches_df = pipeline.process_queries(embs_filenames, top_k=50)

Calculate descriptors:   0%|          | 0/2 [00:00<?, ?it/s]

imsize=768:   0%|          | 0/74 [00:00<?, ?it/s]

imsize=128:   0%|          | 0/74 [00:00<?, ?it/s]

  0%|          | 0/1850 [00:00<?, ?it/s]

KeyboardInterrupt: 